In [1]:
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import warnings
warnings.filterwarnings("ignore")


In [2]:
dataset_dir = pathlib.Path('datasets/flower_photos/')
batch_size = 32
img_width = 180
img_height = 180

In [3]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset='training',
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

valid_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset='validation',
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)


Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.


In [4]:
class_names = train_ds.class_names
print(f'Class names: {class_names}')

Class names: ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


In [5]:
num_classes = len(class_names)
model = Sequential([
    
    layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    
    # аугментация
    layers.experimental.preprocessing.RandomFlip('horizontal', input_shape=(img_height, img_width, 3)), # сл. отражение по горизонтали
    layers.experimental.preprocessing.RandomRotation(0.1), # сл. поворот в пределах 10%
    layers.experimental.preprocessing.RandomZoom(0.1), # сл. зум в пределах 10%
    layers.experimental.preprocessing.RandomContrast(0.2), # сл. изм. контраста

    
    # 3 блока свертки с максимальным объединяющим слоем в каждо
    # и с полносвязным слоем на 128 единиц поверх вместе с нелинейной активац функц relu
    layers.Conv2D(16, 3, padding='same', activation='relu'),  # Note the correct spelling "Conv2D"
    layers.MaxPooling2D(),
    
    layers.Conv2D(32, 3, padding='same', activation='relu'),  # Corrected "Conv2D"
    layers.MaxPooling2D(),
    
    layers.Conv2D(64, 3, padding='same', activation='relu'),  # Corrected "Conv2D"
    layers.MaxPooling2D(),
    
    # регуляризация
    layers.Dropout(0.2),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes),
])

In [6]:
# compile the model

model.compile(optimizer='adam',
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])


In [9]:
# load the model
model.load_weights('my_flowers_model')

# evaluate the model
loss, acc = model.evaluate(train_ds, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))
                        

92/92 - 7s - loss: 0.6613 - accuracy: 0.8011 - 7s/epoch - 81ms/step
Restored model, accuracy: 80.11%
1/1 [==============================] - 0s 105ms/step


In [11]:
#load image
img = tf.keras.utils.load_img('sunflower_test.png', target_size=(img_height, img_width))
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)
                           
# make predictions
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])   

1/1 [==============================] - 0s 19ms/step


In [12]:
# print inference result

print('На изображении скорее всего {} ({:.2f}% вероятность)'.format(
    class_names[np.argmax(score)],
    100 * np.max(score)))

На изображении скорее всего sunflowers (100.00% вероятность)
